# TG Raman process code

<a target="_blank" href="https://colab.research.google.com/github/usnistgov/TGRS-Plasticizer-Library/blob/main/TG%20Raman%20Library%20work%20process.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Declare relevant Python packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.signal import find_peaks
import csv
import requests
from urllib.parse import quote
from itertools import combinations, chain
import math


### Identifies files in folder

In [2]:
local = False

In [19]:
import io, pathlib, functools, time

REC_ID = "mds2-3837"                     # plasticizer library record
META_URL = f"https://data.nist.gov/od/id/{REC_ID}"
API_KEY  = os.getenv("NIST_API_KEY")       # improves rate-limit

# Declare variables
csv_files = []
Access_URLs = []


# Access data from MIDAS repository
sess = requests.Session()
if API_KEY:
    sess.headers["apikey"] = API_KEY      
@functools.lru_cache
# Opens repository
def get_metadata():
    r = sess.get(META_URL, timeout=30)     # Accept: application/json by default
    r.raise_for_status()
    return r.json()

# Identifies the folder 'Raw files' in the repository and extracts file names and storage locations
def list_raw_files_flat(path="Raw files"):
    meta = get_metadata()
    comps = meta.get("components", [])
    for c in comps:
        fp = c.get("filepath", "")
        downloadurl = c.get("downloadURL", "")
        if downloadurl.startswith("https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/"):  # e.g. "Raw files/Filename.xlsx"
            if downloadurl[-7:] != ".sha256": # Issolates files and excludes extraneous files
                continue
            downloadurl = downloadurl[:-7]
            fp = fp[:-7]
            #print(os.path.normpath(fp).split(os.path.sep))
            csv_files.append(os.path.normpath(fp).split(os.path.sep)[1])
            Access_URLs.append(downloadurl)
    return csv_files, Access_URLs

In [20]:
if local:
    myfilter = (
        lambda file: os.path.splitext(file)[1] == ".csv"
    )  # selects only the csv files
    csv_files = sorted([f for f in os.listdir("Raw files") if myfilter(f)])
    path = "Raw files/"

else:
    csv_files, file_locs = list_raw_files_flat("Raw files")
    print(file_locs)
    

['https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/tri-butoxyethyl%20phosphate_07_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/butylbenzyl%20phthalate_26_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/methyl%20palmitate_13_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/di%282-ethylhexyl%29%20phthalate_35_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/diethylene%20glycol%20dibenzoate_02_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/bis%282-hydroxyethyl%29%20dimerate_99_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/dimethyl%20isophthalate_71_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/triisononyl%20trimellitate_17_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%20files/dicapryl%20adipate_41_3D_Raw.csv', 'https://data.nist.gov/od/ds/ark:/88434/mds2-3837/Raw%2

### Declares functions

In [5]:
def Calibration(path, raw_data):
    """ Reads in the calibration shift and performs that calibration of the wavenumber axis

    Parameters:
    path (str): if local = The path + file name, if accessing repository = the download file
    raw_data (pd.dataframe, float64): The unmodified dataframe of data

    Returns:
    cal_data (pd.dataframe, float64): The calibrated data
    """
    # Reads in file to acquire calibration shift
    df = pd.read_csv(path, skiprows=13, nrows=1, header=None, delimiter=":")
    
    Cal_Delta = str(df.iloc[0][1])[
        0:4
    ]  # Pulls out calibration factor from raw data file line 13 accounting for sigfigs
    
    # Wavenumber correction
    raw_data.iloc[:, [0]] += float(Cal_Delta)
    cal_data = raw_data

    return cal_data

In [6]:
def declare_snip(df, StartTG, EndTG, StartWN_snip, EndWN_snip):
    """ declare_snip processes the data including: time-gating, selecting the wavenumber range, and normalizing the data

    Parameters:
    df (pd.dataframe, float64): The calibrated dataframe of data
    StartTG (int, nano seconds): The lower bound of the time-gating window declared in the 'Runs the Code' block
    EndTG (int, nano seconds): The upper bound of the time-gating window declared in the 'Runs the Code' block
    StartWN_snip (int, cm-1): The lower bound of the wavenumber window declared in the 'Runs the Code' block
    EndWN_snip (int, cm-1): The upper bound of the wavenumber window declared in the 'Runs the Code' block

    Returns:
    Target_snip (pd.dataframe,float64): Processed X-Y formatted data as a 1-D dataframe (cm-1,intensity)
    """

    times = list(df.columns.values)
    time_range = np.array(times, dtype=float)

    wave_num = df.index
    wave_num = np.asarray(wave_num, dtype=np.float32)

    # find index location of begining of time gate snip
    for i in range(0, len(time_range)):
        dif_1 = time_range[i] - StartTG
        if dif_1 >= 0:
            start_snip = i + 1  # +1 accounts for python read in eg 1:4 reads in 1,2,3
            break

    # find index location of end of time gate snip
    for i in range(0, len(time_range)):
        dif_2 = time_range[i] - EndTG
        if dif_2 >= 0:
            end_snip = i + 1  # +1 accounts for python read in eg 1:4 reads in 1,2,3
            break

    # TG_snip grabs the time range desired for time gating
    TG_snip = df[df.columns[start_snip:end_snip]]

    # Target_snip is a dataframe with xy data, (wavenumber,averaged intensity data for snip region)
    Target_snip = TG_snip.mean(axis=1)

    # Rename intensity column
    Target_snip.name = "intensity"

    # find location of start of wavenumber snip
    for i in range(0, len(wave_num)):
        dif_3 = wave_num[i] - StartWN_snip
        if dif_3 >= 0:
            WNstart_snip = (
                i + 1
            )  # time_range.index(i)+1 # +1 accounts for python read in eg 1:4 reads in 1,2,3
            break
    # find location of end of wavenumber snip
    for i in range(0, len(wave_num)):
        dif_4 = wave_num[i] - EndWN_snip
        if dif_4 >= 0:
            WNend_snip = (
                i + 1
            )  # time_range.index(i)+1 # +1 accounts for python read in eg 1:4 reads in 1,2,3
            break

    Target_snip = Target_snip.iloc[WNstart_snip:WNend_snip]

    # Normalize Data
    Target_snip = Target_snip / Target_snip.abs().max()

    return Target_snip #, StartTG, EndTG

In [7]:
def Spectra_plot(df, sample_name):
    """ Generates a folder named "plots" and plots the 2-D Raman spectra for each sample (.png)

    Parameters:
    df (pd.dataframe, float64): The processed 1-D spectral Raman data
    sample_name (str): The file name

    Returns:
    path: path of the generated plot png
    """
    # Plotting
    fig, ax = plt.subplots(1, 1)
    ax.plot(df)
    ax.set_title(
        os.path.splitext(sample_name[:-11])[0]
    )  # Sets the title of plot to title of the csv file
    plt.xlabel(
        "Raman shift ($\\mathregular{cm^{-1}}$)", labelpad=2
    )  # labelpad=10 adjusts space between axis lable and plot
    plt.ylabel("Intensity", labelpad=10)

    # PLOT AESTHETICS
    # set font sizes
    S_size = 12
    M_size = 14
    L_size = 16

    plt.rc("font", size=S_size)  # controls default text sizes
    plt.rc("axes", titlesize=M_size)  # fontsize of the axes title
    plt.rc("axes", labelsize=M_size)  # fontsize of the x and y labels
    plt.rc("xtick", labelsize=S_size)  # fontsize of the tick labels
    plt.rc("ytick", labelsize=S_size)  # fontsize of the tick labels
    plt.rc("legend", fontsize=S_size)  # legend fontsize
    plt.rc("figure", titlesize=L_size)  # fontsize of the figure title

    # Export plots
    # next 2 lines create a new folder in current folder to store plots
    directory = "plots"
    os.makedirs(directory, exist_ok=True)
    filename = os.path.splitext(sample_name)[0] + ".png"
    path = os.path.join(directory, filename)
    fig.savefig(path)
    plt.close()

    return path

In [8]:
# Calculates the spectral background
# derived from stackoverflow code written for python 3.6:  https://stackoverflow.com/questions/29156532/python-baseline-correction-library -- Source: "Baseline Correction with Asymmetric Least Squares Smoothing" Paul H. C. Eilers Hans F.M. Boelens. Download access at- https://prod-dcd-datasets-public-files-eu-west-1.s3.eu-west-1.amazonaws.com/dd7c1919-302c-4ba0-8f88-8aa61e86bb9d

# y is the signal of length L, 
# 
# D is the difference matrix assuming a second order difference matrix
# w is a weighted vector (currently un weighted e.g. all ones) if peak regions are known, can be set to zero in those regions adjusted in last boolean statment over iterations
# W is a diagonal matrix with w on its diagonal, lam adjusts the balance between terms
# p is the asymmetry parameter - recommended set between 0.001-0.1
# lam: 10^4 to 10^6 are a good starting point

def baseline_als(y, lam, p, niter=10):
    """ Calculates the background fit using and alternating least squares fit

    Paramters:
    y (pd.dataframe, float64): The processed 1-D spectral Raman data, 
    lam (float64): Adjusts the balance of terms in W. 10^4 to 10^6 are a good starting point
    p (float64): The asymmetry parameter - recommended set between 0.001-0.1
    niter (int): The iteration counter
    D (scipy sparse diagonal matrix): Is the difference matrix assuming a second order difference matrix
    w (np.array) Is a weighted vector of length y, if peak regions are known, can be set to zero in those regions adjusted in last boolean statment over iterations
    W  (): Is a diagonal matrix with w on its diagonal, lam adjusts the balance between terms

    Returns:
    z (pd.dataframe,float64): is the fitted baseline
    peakfind (pd.series, int32): A 1-D array of length y indicating if the equivalent index in y is a peak (0s) or not a peak (1s)
    """
    L = len(y)
    # constructs a sparse matrix from diagonals
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    w = np.ones(L)
    i = 0
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        # z solves for the optimization of the statement with Z intermediate
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z.tocsc(), w * y)
        w = p * (y > z) + (1 - p) * (y < z)
        # peakfind identifies where peaks are (assigned zero value) and where the absence of peaks (assigned 1 value)
        # sensitivity value 0.02 was tuned using the level of noise in a test image and may need to be adjusted
        peakfind = (y - z > 0.02) * 0 + (y - z <= 0.02) * 1

    return z, peakfind

In [9]:
def run_length(vals, max_length=500):
    """ Determines the the longest run with no peaks. Used in the SBR calcualtion

    Parameters:
    vals (pd.series, int32): an array of 1s and 0s the same length as the processed 1-D Raman data indicating where there 
        are peaks (0s) and where there are no peaks (1s). Calculated in baseline_als (variable name, peakfind).
    max_length (int): A cut off condition if there are long runs with no peaks

    Returns:
    best_length (,int): the longest run with no peaks
    range_end (int): the index
    """
    best_length = 0
    current_length = 0
    i = 0
    range_end = 0
    for i, t in enumerate(vals):
        if t == 1:
            current_length += 1
            if current_length > best_length:
                range_end = i
                best_length = current_length
            if current_length == max_length:
                range_end = i
                best_length = max_length
                break
        else:
            current_length = 0

    return best_length, range_end

In [10]:
def SBRcalc(df):
    """ Calculates the Signal-to Backgroun Ratio (SBR)
    Parameters:
    df (pd.dataframe, float64): The processed 1-D spectral Raman data

    Returns:
    sbr (int): The SBR for the current sample
    """
    base, ispeak = baseline_als(df, 10e5, 0.01, niter=15)
    # background subtracted data - to determine height of tallest peak above fitted background
    y = df - base

    # find a long run with no peaks
    min_length = 100
    troughs = ispeak.values
    best_length, range_end = run_length(troughs, max_length=500)

    if best_length < min_length:
        # if there is not sufficient region found to have 'no peaks' it is assumed that
        # the range is primarily background or broad peaks where the noise can be accessible without too many peak outliers
        run = df
    else:
        range_begin = range_end - best_length + 50
        no_peaks = df.iloc[range_begin:range_end]
        run = no_peaks

    fit, toss = baseline_als(run, 10e4, 0.5, niter=15)
    flat_noise = run - fit

    error_STD = flat_noise.std()
    # SBR = height above baseline of max peak / STD of background --- 2*STD =95% confidence interval
    sbr = y.max() / (2 * error_STD)
    sbr = round(sbr)

    return sbr

In [11]:
# Exports text data and metadata details:
def details_CSV(df, sample_name, StartTG_snip, EndTG_snip, sbr):
    """ Creates a folder named "Final Processed Files" and generates CSV files containing the processed data and metadata

    Parameters:
    df (pd.dataframe, float64): The processed 1-D spectral Raman data
    sample_name (str): The file name
    StartTG_snip (int, nano seconds): The lower bound of the time-gating window declared in the 'Runs the Code' block
    EndTG_snip (int, nano seconds): The upper bound of the time-gating window declared in the 'Runs the Code' block
    sbr (int): The signal to background ratio

    """

    # Rename DataFrame index for data print
    df = df.rename_axis('WaveNumber')
    df.name = "Intensity"

    # Adjust name
    Read_in_Name = os.path.splitext(sample_name)[0]
    Sample_Name = Read_in_Name[:-7]
    # writing metadata to csv
    directory = "Final_Processed_Files"
    os.makedirs(directory, exist_ok=True)
    filename = Sample_Name + "_Proc" + ".csv"
    path = os.path.join(directory, filename)

    # Metadata content options
    # 1. sample name
    name = "Sample: " + Sample_Name + "\n"
    # 2. normalization protocol
    norm_proc = "Intensity data was averaged across the time gated region (df.mean(axis=1)) followed by normalization by dividing the whole range by the absolute value of the highest point\n"
    # 3. gating snip
    snip_range = (
        "The time gated region is "
        + str(StartTG_snip)
        + " - "
        + str(EndTG_snip)
        + " ns\n"
    )
    # 4. Signal to background -reference to self
    SBR_read = f"The signal to background ratio (SBR) is: {sbr}\n"
    # 5. Output Units
    Units = "WaveNumber is in units cm-1 and Intensity is in arbitrary units (A.U.)\n"

    # open file to write
    # 'w' indicates that the file is open to 'write' mode
    with open(path, "w") as csvfile:
        # creating a csv writer object
        csvwriter = csv.writer(csvfile)

        # Print to file
        # 1. sample name
        csvfile.write(name)
        # 2. normalization protocol
        csvfile.write(norm_proc)
        # 3. gating snip
        csvfile.write(snip_range)
        # 4. Signal to background -reference to self
        csvfile.write(SBR_read)
        # 5. Print Units
        csvfile.write(Units)

    # insert xy data
    df.to_csv(path, mode="a", sep=";")

    path = os.path.join(directory, filename)

In [29]:
def Data_reader(path, start_snip, end_snip, StartWN_snip, EndWN_snip):
    """ Reads in the CSV data and initiates calibration

    Parameters:
    path (str) : if local = path + file name, if accessing repository = the file download path
    start_snip (int, nano seconds): The lower bound of the time-gating window declared in the 'Runs the Code' block
    end_snip (int, nano seconds): The upper bound of the time-gating window declared in the 'Runs the Code' block
    StartWN_snip (int, cm-1): The lower bound of the wavenumber window declared in the 'Runs the Code' block
    EndWN_snip (int, cm-1): The upper bound of the wavenumber window declared in the 'Runs the Code' block

    Returns:
    Target_snip (pd.dataframe,float64): Processed X-Y formatted data as a 1-D dataframe (cm-1,intensity)
    """
    # Reads raw time gate Raman data into a Pandas dataframe
    df = pd.read_csv(path, sep=";", skiprows=15) 
    
    # Wavenumber calibration
    df = Calibration(path, df)

    # Renames first column and set to index
    df.rename(columns={"NaN": "cm-1"}, inplace=True)
    df.set_index("cm-1", inplace=True, drop=True)

    # declare_snip snips data to declared gating region and wavenumber window, averages it to 2D spectra and normalizes data between 0-1 intensity

    Target_snip = declare_snip(
        df, start_snip, end_snip, StartWN_snip, EndWN_snip
    )  # update in format of declare_snip(StartTG_snip,EndTG_snip) in nanoseconds

    return Target_snip

### Runs the code

In [30]:
# Declare the desired time-gating region -- (5.5-5.8) ns works well for the plasticizer data library
# Using a larger gating range captures more fluorescence and simulates what might be seen from a conventional, continuous wave Raman spectrometer.
StartTG_snip = 5.5
EndTG_snip = 5.8

# Declare wavenumber snip region - the function will identify the index of values closest to the input values
StartWN_snip = (
    130  # Full available window is 127-2515 cm-1 +/- 3 cm-1 
)
EndWN_snip = 2500

current_spectra = []

if local == True:
    for f in csv_files:
    # Averages over the snipped region, normalizes data, and returns
        current_spectra = Data_reader(path + f, StartTG_snip, EndTG_snip, 
        StartWN_snip, EndWN_snip
        )
        # Creates a new folder called Plots and creates PNG plots of all the spectra in the folder
        Spectra_plot(current_spectra, f)
        # Calculates Signal-to-Background Ratio
        SBR = SBRcalc(current_spectra)
        # Creates a new folder called Final Processed Files and creates csv files including processed, gated Raman data and the SBR
        details_CSV(current_spectra, f, StartTG_snip, EndTG_snip, SBR)
else:
    # Reads in the csv files one at a time and isolates the snipped region, adjusts wavenumbers for instrument calibration error
    for f, loc in zip(csv_files, file_locs):
        # Averages over the snipped region, normalizes data, and returns
        current_spectra = Data_reader(loc, 
        StartTG_snip,
        EndTG_snip,
        StartWN_snip,
        EndWN_snip,
        )
        # Creates a new folder called Plots and creates PNG plots of all the spectra in the folder
        Spectra_plot(current_spectra, f)
        # Calculates Signal-to-Background Ratio
        SBR = SBRcalc(current_spectra)
        # Creates a new folder called Final Processed Files and creates csv files including processed, gated Raman data and the SBR
        details_CSV(current_spectra, f, StartTG_snip, EndTG_snip, SBR)

HTTPError: HTTP Error 502: Bad Gateway

### Calculates the SNRs and stores them as an array of paired chemistries and SNRs

$$
\text{SNR}=\frac{Y_{avg}}{\left(\sigma_{noise}/\sqrt{2}\right)}
$$

Where $\text{Y}_{avg}$ is an average of the peak height above background of the highest-intensity Raman band across successive replicates. $\sigma_{noise}$ is the average of the standard deviation (STD) of the noise calculated under the footprint of the highest-intensity Raman band from each replicate. 

In [ ]:
# Declare empty list
SNR_list = []

# Find all the unique chemistries
chemistries = [f[:-14] for f in csv_files]
unique_chem, unique_indicies, unique_counts = np.unique(chemistries, return_index=True, return_counts=True)

# Loop through each chemistry
for i, chem in enumerate(unique_chem):
    replicalist = [f for f in csv_files if chem == f[:-14]]
    peak_list = []
    Noise_list = []
    STD_list = []

    # Finds the pairs and round-robin calculates components for the SNR calculation
    for pair in combinations(replicalist, 2):
        
        spectraA = pd.read_csv('Final_Processed_Files/'+pair[0].replace('3D_Raw','Proc'), sep=";", skiprows=5).iloc[:]
        spectraA.set_index(spectraA.columns[0], inplace = True)
        spectraB = pd.read_csv('Final_Processed_Files/'+pair[1].replace('3D_Raw','Proc'), sep=";", skiprows=5).iloc[:]
        spectraB.set_index(spectraB.columns[0], inplace = True)

        #Cutting spectra at 400-2500 cm-1 to cut out peaks in the Rayleigh wing
        spectraA = spectraA[spectraA.index >= 400]
        spectraB = spectraB[spectraB.index >= 400]
        
        # Performs background subtraction on the current spectra and previous spectra if sample name is the same compound
        baseA, peaksA = baseline_als(spectraA.iloc[:,0], 10e5, 0.01, niter=15)
        baseB, peaksB = baseline_als(spectraB.iloc[:,0], 10e5, 0.01, niter=15) 
        specA_bkgsub = spectraA.iloc[:,0] - baseA
        specB_bkgsub = spectraB.iloc[:,0] - baseB       

        # Finds major peak in the spectra and selects a range of 24 wavenumber about the peak
        peaks, propertiesA = find_peaks(
            spectraA.iloc[:,0], height=0.4
        )   
        max_peak_indexA = np.argmax(np.array(propertiesA["peak_heights"]))
        max_peakA = int(peaks[max_peak_indexA])

        # Extracts the wavenumber range from the dataframe index
        bumped = specA_bkgsub.reset_index()
        wavenumA = bumped.iloc[:, 0]
        S_RangeA = wavenumA.iloc[max_peakA] - 12
        E_RangeA = wavenumA.iloc[max_peakA] + 12

        # Finds major peak in the spectra and selects a range of 24 wavenumber about the peak
        peaks, propertiesB = find_peaks(
            spectraB.iloc[:,0], height=0.4
        )  # finds the most significant peak in the spectra.
        max_peak_indexB = np.argmax(np.array(propertiesB["peak_heights"]))
        max_peakB = int(peaks[max_peak_indexB])

        bumped = specB_bkgsub.reset_index()
        wavenumB = bumped.iloc[:, 0]
        S_RangeB = wavenumB.loc[max_peakB] - 12
        E_RangeB = wavenumB.loc[max_peakB] + 12
        
        # Subtracts the baseline corrected spectra of successive replicates within the region about the major peak
        Peak_snipA = specA_bkgsub.loc[S_RangeA:E_RangeA].to_numpy()
        Peak_snipB = specB_bkgsub.loc[S_RangeB:E_RangeB].to_numpy()
        Noise = np.subtract(Peak_snipA, Peak_snipB)
        
        # Calculates standard deviation of the noise in the region under the major peak and calculates average height of the major peak between successive spectra
        peakA = specA_bkgsub.max()
        peakB = specB_bkgsub.max()
        peaks = [peakA, peakB]
        
        peak_list.append(peaks)
        Noise_list.append(Noise)
        
    flat_peaks = list(chain.from_iterable(peak_list))
    peak_ave = sum(flat_peaks)/len(flat_peaks)

    STD_list = [np.std(noise) for noise in Noise_list]
    avg_STD = math.sqrt(sum(x**2 for x in STD_list)/len(STD_list))
    
    # Calculates SNR based on McCreery 2000 Chap. 4
    SNR = peak_ave / (avg_STD / np.sqrt(2))
    SNR = round(SNR)
    SNR_list.append(SNR)

Details = pd.DataFrame({'Names':unique_chem, "SNR": SNR_list})

    